# Platform Utilities

> Cross-platform utilities for process management, path handling, and system detection

In [ ]:
#| default_exp core.platform

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import platform
import subprocess
import json
from pathlib import Path
from typing import Dict, Any, Optional

This module provides cross-platform utilities to support Linux, macOS, and Windows:

| Category | Functions |
|----------|----------|
| **Detection** | `is_windows()`, `is_macos()`, `is_linux()`, `is_apple_silicon()`, `get_current_platform()` |
| **Paths** | `get_python_in_env()` |
| **Process** | `get_popen_isolation_kwargs()`, `terminate_process()`, `terminate_self()` |
| **Shell** | `run_shell_command()`, `conda_env_exists()` |

## Platform Detection

Functions to detect the current operating system and architecture.

In [ ]:
#| export
def is_windows() -> bool:
    """Check if running on Windows."""
    return platform.system() == "Windows"


def is_macos() -> bool:
    """Check if running on macOS."""
    return platform.system() == "Darwin"


def is_linux() -> bool:
    """Check if running on Linux."""
    return platform.system() == "Linux"


def is_apple_silicon() -> bool:
    """Check if running on Apple Silicon Mac (for MPS detection)."""
    return is_macos() and platform.machine() == "arm64"

In [ ]:
# Test detection functions
print(f"is_windows(): {is_windows()}")
print(f"is_macos(): {is_macos()}")
print(f"is_linux(): {is_linux()}")
print(f"is_apple_silicon(): {is_apple_silicon()}")

# Exactly one of these should be True
assert sum([is_windows(), is_macos(), is_linux()]) == 1

is_windows(): False
is_macos(): False
is_linux(): True
is_apple_silicon(): False


In [ ]:
#| export
def get_current_platform() -> str:
    """Get current platform string for manifest filtering.
    
    Returns strings like 'linux-x64', 'darwin-arm64', 'win-x64'.
    """
    system = platform.system().lower()
    machine = platform.machine().lower()
    
    # Normalize system names
    if system == "darwin":
        pass  # Keep as darwin
    elif system == "windows":
        system = "win"
    
    # Normalize architecture
    if machine in ("x86_64", "amd64"):
        arch = "x64"
    elif machine in ("arm64", "aarch64"):
        arch = "arm64"
    else:
        arch = machine
    
    return f"{system}-{arch}"

In [ ]:
# Test platform string
current = get_current_platform()
print(f"Current platform: {current}")

# Should be one of the expected formats
valid_prefixes = ["linux-", "darwin-", "win-"]
assert any(current.startswith(p) for p in valid_prefixes)

Current platform: linux-x64


## Path Utilities

Functions for cross-platform path handling, particularly for conda environments.

In [ ]:
#| export
def get_python_in_env(
    env_path: Path  # Path to conda environment root
) -> Path:  # Path to Python executable
    """Get the Python executable path for a conda environment.
    
    On Windows: env_path/python.exe
    On Unix: env_path/bin/python
    """
    if is_windows():
        return env_path / "python.exe"
    else:
        return env_path / "bin" / "python"

In [ ]:
# Test path generation
test_env = Path("/home/user/miniforge3/envs/test-env")
python_path = get_python_in_env(test_env)
print(f"Python path: {python_path}")

if is_windows():
    assert python_path.name == "python.exe"
else:
    assert "bin" in python_path.parts
    assert python_path.name == "python"

Python path: /home/user/miniforge3/envs/test-env/bin/python


## Process Management

Cross-platform utilities for subprocess creation and termination.

In [ ]:
#| export
def get_popen_isolation_kwargs() -> Dict[str, Any]:
    """Return kwargs for process isolation in subprocess.Popen.
    
    On Unix: Returns {'start_new_session': True}
    On Windows: Returns {'creationflags': CREATE_NEW_PROCESS_GROUP}
    
    Usage:
        process = subprocess.Popen(cmd, **get_popen_isolation_kwargs(), ...)
    """
    if is_windows():
        # CREATE_NEW_PROCESS_GROUP allows the process to be terminated
        # without affecting the parent process
        return {"creationflags": subprocess.CREATE_NEW_PROCESS_GROUP}
    else:
        # start_new_session creates a new process group on Unix
        return {"start_new_session": True}

In [ ]:
# Test isolation kwargs
kwargs = get_popen_isolation_kwargs()
print(f"Isolation kwargs: {kwargs}")

if is_windows():
    assert "creationflags" in kwargs
else:
    assert kwargs.get("start_new_session") == True

Isolation kwargs: {'start_new_session': True}


In [ ]:
#| export
def terminate_process(
    process: subprocess.Popen,  # Process to terminate
    timeout: float = 2.0  # Seconds to wait before force kill
) -> None:
    """Terminate a subprocess gracefully, with fallback to force kill.
    
    On all platforms:
    1. Calls process.terminate() (SIGTERM on Unix, TerminateProcess on Windows)
    2. Waits for timeout seconds
    3. If still running, calls process.kill() (SIGKILL on Unix, TerminateProcess on Windows)
    """
    if process is None or process.poll() is not None:
        return  # Already terminated
    
    process.terminate()
    try:
        process.wait(timeout=timeout)
    except subprocess.TimeoutExpired:
        process.kill()
        process.wait()  # Reap the process

In [ ]:
#| export
def terminate_self() -> None:
    """Terminate the current process (for worker suicide pact).
    
    On Unix: Sends SIGTERM to self for graceful shutdown
    On Windows: Calls os._exit() since Windows lacks SIGTERM
    """
    if is_windows():
        # Windows doesn't have SIGTERM, use os._exit for immediate termination
        # Exit code 1 indicates abnormal termination
        os._exit(1)
    else:
        import signal
        os.kill(os.getpid(), signal.SIGTERM)

## Shell Command Execution

Cross-platform shell command execution without hardcoded shell paths.

In [ ]:
#| export
def run_shell_command(
    cmd: str,  # Shell command to execute
    check: bool = True,  # Whether to raise on non-zero exit
    capture_output: bool = False,  # Whether to capture stdout/stderr
    **kwargs  # Additional kwargs passed to subprocess.run
) -> subprocess.CompletedProcess:
    """Run a shell command cross-platform.
    
    Unlike using shell=True with executable='/bin/bash', this function
    uses the platform's default shell:
    - Linux/macOS: /bin/sh (or $SHELL)
    - Windows: cmd.exe
    """
    print(f"Running: {cmd}")
    return subprocess.run(
        cmd,
        shell=True,
        check=check,
        capture_output=capture_output,
        **kwargs
    )

In [ ]:
#| export
def conda_env_exists(
    env_name: str,  # Name of the conda environment
    conda_cmd: str = "conda"  # Conda command (conda, mamba, micromamba)
) -> bool:
    """Check if a conda environment exists (cross-platform).
    
    Uses 'conda env list --json' instead of piping to grep,
    which doesn't work on Windows.
    """
    try:
        result = subprocess.run(
            [conda_cmd, "env", "list", "--json"],
            capture_output=True,
            text=True
        )
        if result.returncode != 0:
            return False
        
        data = json.loads(result.stdout)
        # Extract env names from paths
        for path in data.get('envs', []):
            if Path(path).name == env_name:
                return True
        return False
    except (subprocess.SubprocessError, json.JSONDecodeError, FileNotFoundError):
        return False

In [ ]:
# Test shell command (simple echo)
if is_windows():
    result = run_shell_command("echo hello", capture_output=True)
else:
    result = run_shell_command("echo hello", capture_output=True)

print(f"Return code: {result.returncode}")
print(f"Output: {result.stdout}")
assert result.returncode == 0

Running: echo hello
Return code: 0
Output: b'hello\n'


In [ ]:
# Test conda_env_exists (will return False for non-existent env)
exists = conda_env_exists("this-env-should-not-exist-12345")
print(f"Non-existent env exists: {exists}")
assert exists == False

# Test with base env "miniforge3" (should exist if miniforge3 is installed)
base_exists = conda_env_exists("miniforge3")
print(f"Base env exists: {base_exists}")

Non-existent env exists: False
Base env exists: True


## Summary

This module provides the following cross-platform utilities:

### Detection
- `is_windows()`, `is_macos()`, `is_linux()` - OS detection
- `is_apple_silicon()` - Apple Silicon detection for MPS
- `get_current_platform()` - Platform string like "linux-x64"

### Paths
- `get_python_in_env(env_path)` - Python executable path in conda env

### Process Management
- `get_popen_isolation_kwargs()` - Kwargs for subprocess isolation
- `terminate_process(process, timeout)` - Graceful process termination
- `terminate_self()` - Self-termination for suicide pact

### Shell Commands
- `run_shell_command(cmd, ...)` - Cross-platform shell execution
- `conda_env_exists(env_name)` - Check if conda env exists

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()